In [1]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def avg_pool_2x2(x):
    return tf.nn.avg_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

def conv_layer(scope, x, filter_dim, pool = 'max', use_lrn = True, reshape_dim_list = None):
    with tf.variable_scope(scope):
        W_conv1 = tf.get_variable('wconv1', filter_dim, initializer = tf.truncated_normal_initializer(stddev = 0.02))
        b_conv1 = tf.get_variable('wconv2', filter_dim[-1], initializer = tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1) if pool == 'max' else avg_pool_2x2(h_conv1)
        if use_lrn:
            h_pool1 = tf.nn.local_response_normalization(h_pool1)
        if reshape_dim_list:
            h_pool1 = tf.reshape(h_pool1, reshape_dim_list)
        return h_pool1
    
def fc_layer(scope, x, weight_shape, activation = 'relu', dropout_prob = None):
    with tf.variable_scope(scope):
        W_fc1 = tf.get_variable('wfc', weight_shape, initializer = tf.truncated_normal_initializer(stddev = 0.02))
        b_fc1 = tf.get_variable('bfc', weight_shape[-1], initializer = tf.constant_initializer(0))
        h_fc1 = tf.matmul(x, W_fc1) + b_fc1
        if activation == 'relu': 
            h_fc1 = tf.nn.relu(h_fc1)
        elif activation == 'tanh':
            h_fc1 = tf.nn.tanh(h_fc1)
        if dropout_prob:
            h_fc1 = tf.nn.dropout(h_fc1, keep_prob = 1 - dropout_prob)
        return h_fc1


In [4]:
def discriminator(x_image, reuse=False):
    if (reuse):
        tf.get_variable_scope().reuse_variables()
    #First Conv and Pool Layers
    h_pool1 = conv_layer('d-conv1-layer', x_image, [5,5,1,8])
    h_pool2_flat = conv_layer('d-conv2-layer', h_pool1, [5,5,8,16], reshape_dim_list = [-1, 7*7*16])
    h_fc1 = fc_layer('d-fc1-layer', h_pool2_flat, [7 * 7 * 16, 32], dropout_prob = 0.5)
    h_fc2 = fc_layer('d-fc2-layer', h_fc1, [32, 1], activation = None)
    return h_fc2

In [5]:
def deconv_layer(scope, x, output_shape, kernel_dim, strides = [1,2,2,1], padding = 'SAME', activation = 'relu'):
        with tf.variable_scope(scope):
            W_conv1 = tf.get_variable('wconv1', [kernel_dim, kernel_dim, output_shape[-1], int(x.get_shape()[-1])], 
                                      initializer = tf.truncated_normal_initializer(stddev=0.1))
            b_conv1 = tf.get_variable('bconv1', [output_shape[-1]], initializer = tf.constant_initializer(.1))
            h_conv1 = tf.nn.conv2d_transpose(x, W_conv1, output_shape = output_shape, strides = strides, 
                                             padding = padding) + b_conv1
            h_conv1 = tf.contrib.layers.batch_norm(inputs = h_conv1, center=True, scale=True, 
                                                   is_training=True, scope = 'batch-norm')
            if activation == 'relu':
                h_conv1 = tf.nn.relu(h_conv1)
            else: # only support relu/tanh for now
                h_conv1 = tf.nn.tanh(h_conv1)
            return h_conv1

In [6]:
def generator(z, batch_size, z_dim, reuse=False):
    if (reuse):
        tf.get_variable_scope().reuse_variables()
    g_dim = 64 # of filters
    c_dim = 1 # color dim (use 3 if rgb)
    s = 28 #square dim size of the image
    output1_shape = [batch_size, int(s/8), int(s/8), 4 * g_dim]
    output2_shape = [batch_size, int(s/4) - 1, int(s/4) - 1, 2 * g_dim]
    output3_shape = [batch_size, int(s/2) - 2, int(s/2) - 2, g_dim]
    output4_shape = [batch_size, s, s, c_dim]

    h0 = tf.reshape(z, [batch_size, int(s/16) +1, int(s/16) +1, 25])
    h0 = tf.nn.relu(h0)
    h_conv1 = deconv_layer('g-conv-1', h0, output1_shape, 5)
    h_conv2 = deconv_layer('g-conv-2', h_conv1, output2_shape, 5)
    h_conv3 = deconv_layer('g-conv-3', h_conv2, output3_shape, 5)
    h_conv4 = deconv_layer('g-conv-4', h_conv3, output4_shape, 5, padding = 'VALID', activation = 'tanh')
    return h_conv4

In [7]:
def show_generated_image():
    z_dim = 100
    z_holder = tf.placeholder(tf.float32, [None, z_dim])
    sample_img = generator(z_holder, 1, z_dim)
    test_z = np.random.normal(-1, 1, [1, z_dim])
    img = sample_img.eval(feed_dict = {z_holder: test_z})
    img = img.squeeze()
    plt.imshow(img, cmap = 'gray_r')
    plt.show()

In [8]:
batch_size = 16
z_dimensions = 100

x_placeholder = tf.placeholder("float", shape = [None,28,28,1]) #Placeholder for input images to the discriminator
z_placeholder = tf.placeholder(tf.float32, [None, z_dimensions]) #Placeholder for input noise vectors to the generator
Dx = discriminator(x_placeholder) #Dx will hold discriminator prediction probabilities for the real MNIST images
Gz = generator(z_placeholder, batch_size, z_dimensions) #Gz holds the generated images
Dg = discriminator(Gz, reuse=True) #Dg will hold discriminator prediction probabilities for generated images

g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Dg, labels = tf.ones_like(Dg))) # ensure forward compatibility: function needs to have logits and labels args explicitly used
d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Dx, labels = tf.ones_like(Dx)))
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = Dg, labels = tf.zeros_like(Dg)))
d_loss = d_loss_real + d_loss_fake

tvars = tf.trainable_variables()
d_vars = [var for var in tvars if 'd-' in var.name]
g_vars = [var for var in tvars if 'g-' in var.name]

with tf.variable_scope(tf.get_variable_scope(),reuse=False): 
    trainerD = tf.train.AdamOptimizer().minimize(d_loss, var_list=d_vars)
    trainerG = tf.train.AdamOptimizer().minimize(g_loss, var_list=g_vars)
    


In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
epochs = 10000
for i in range(epochs):
    z_batch = np.random.normal(-1, 1, size=[batch_size, z_dimensions])
    real_image_batch = mnist.train.next_batch(batch_size)
    real_image_batch = np.reshape(real_image_batch[0],[batch_size,28,28,1])
    _,dLoss = sess.run([trainerD, d_loss],feed_dict={z_placeholder:z_batch,x_placeholder:real_image_batch}) #Update the discriminator
    _,gLoss = sess.run([trainerG,g_loss],feed_dict={z_placeholder:z_batch}) #Update the generator 
    if i % 500 == 0:
        print("discriminator loss: {}".format(dLoss))
        print("generator loss: {}".format(gLoss))

discriminator loss: 1.3862028121948242
generator loss: 0.6936384439468384
discriminator loss: 0.5373177528381348
generator loss: 2.3342442512512207
discriminator loss: 0.11401794850826263
generator loss: 3.5449466705322266


In [ ]:
show_generated_image()
